In [1]:
import os
import re
from glob import glob
import time
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from torchmetrics.classification import MulticlassMatthewsCorrCoef
import open3d as o3
# from open3d import JVisualizer # For Colab Visualization
from open3d.web_visualizer import draw # for non Colab

import matplotlib.pyplot as plt
%matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
#torch geometric could be used to import .ply straight into torch. But it doesn't seem to work with the code.
#import torch_geometric

In [3]:
# TEMP for supressing pytorch user warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
from point_net import PointNetSegHead

In [5]:
# dataset
ROOT = r'./../../data/S3DIS_aligned_reduced_partitioned'

# feature selection hyperparameters
NUM_TRAIN_POINTS = 4096 # train/valid points
NUM_TEST_POINTS = 15000

BATCH_SIZE = 8

In [6]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [7]:
CATEGORIES = {
    'ceiling'  : 0, 
    'floor'    : 1, 
    'wall'     : 2, 
    'beam'     : 3, 
    'column'   : 4, 
    'window'   : 5,
    'door'     : 6, 
    'table'    : 7, 
    'chair'    : 8, 
    'sofa'     : 9, 
    'bookcase' : 10, 
    'board'    : 11,
    'stairs'   : 12,
    'clutter'  : 13
}

# unique color map generated via
# https://mokole.com/palette.html
COLOR_MAP = {
    0  : (47, 79, 79),    # ceiling - darkslategray
    1  : (139, 69, 19),   # floor - saddlebrown
    2  : (34, 139, 34),   # wall - forestgreen
    3  : (75, 0, 130),    # beam - indigo
    4  : (255, 0, 0),     # column - red 
    5  : (255, 255, 0),   # window - yellow
    6  : (0, 255, 0),     # door - lime
    7  : (0, 255, 255),   # table - aqua
    8  : (0, 0, 255),     # chair - blue
    9  : (255, 0, 255),   # sofa - fuchsia
    10 : (238, 232, 170), # bookcase - palegoldenrod
    11 : (100, 149, 237), # board - cornflower
    12 : (255, 105, 180), # stairs - hotpink
    13 : (0, 0, 0)        # clutter - black
}

v_map_colors = np.vectorize(lambda x : COLOR_MAP[x])

NUM_CLASSES = len(CATEGORIES)

In [8]:
#S3dis dataloader stuffs

In [9]:
from torch.utils.data import DataLoader
from s3dis_dataset import S3DIS

# get datasets
s3dis_test = S3DIS(ROOT, area_nums='6', split='test', npoints=NUM_TEST_POINTS)

# get dataloaders
test_dataloader = DataLoader(s3dis_test, batch_size=BATCH_SIZE, shuffle=False)

## object selection testing

In [ ]:
#extracting one colour (and therefore one class) from the point cloud, awesome method I know

pcm = o3.geometry.PointCloud()
pcm.points = o3.utility.Vector3dVector(points.to('cpu')[2, :, :])
pcm.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(pred_choice.to('cpu')[2, :])).T/255)

pcr = o3.geometry.PointCloud()
print(pcm)
print(pcm.colors[0])
colour = str(pcm.colors[1])

#pcr.points = pcm.points[1:100]
for p in range(15000):
    if (str(pcm.colors[p]) == colour):
        pcr.points.append( pcm.points[p])
        

# draw(pcd)
o3.visualization.draw_plotly([pcr])

In [ ]:
#extracting one colour (and therefore one class) from the point cloud, better method hopefully

pcm = o3.geometry.PointCloud()
pcm.points = o3.utility.Vector3dVector(points.to('cpu')[2, :, :])
pcm.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(pred_choice.to('cpu')[2, :])).T/255)

pcr = o3.geometry.PointCloud()
print(pcm)
print(pcm.colors[0])
colour = str(pcm.colors[1])

#pcr.points = pcm.points[1:100]
for p in range(15000):
    if (str(pcm.colors[p]) == colour):
        pcr.points.append( pcm.points[p])
        

# draw(pcd)
o3.visualization.draw_plotly([pcr])

### segmenting a S3dis room

In [ ]:
MODEL_PATH = './trained_models/seg_focal_dice_iou_rot/seg_model_68.pth'

model = PointNetSegHead(num_points=NUM_TEST_POINTS, m=NUM_CLASSES).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval();

In [20]:
torch.cuda.empty_cache() # release GPU memory
points, targets = s3dis_test.get_random_partitioned_space()

In [21]:
print(points.shape)

torch.Size([25, 15000, 3])


In [33]:
# display true full point cloud
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(points.permute(2, 0, 1).reshape(3, -1).to('cpu').T)
pcd.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(targets.reshape(-1).to('cpu'))).T/255)

In [ ]:
#display in Jupyter
draw(pcd)

In [34]:
#display in window
o3.visualization.draw_geometries([pcd]) 

In [25]:
print(targets.shape)

torch.Size([25, 15000])


In [37]:
print(targets[3][1])

if (targets[3][1] == 3):
    print("yes!")

tensor(3)
yes!


In [53]:
#selecting a single class from the image
e_points = np.zeros((25, 15000, 3))
pcr = torch.from_numpy(e_points).type(torch.float32)
print(pcr.shape)

#pcr.points = pcm.points[1:100]
for o in range(25):
    for p in range(15000):
        if (targets[o][p] == 8):
            pcr[o][p] = points[o][p]


torch.Size([25, 15000, 3])


In [68]:
#selecting a single class from the image
e_points = np.zeros((25, 0, 3))
pcr = torch.from_numpy(e_points).type(torch.float32)

#pcr.points = pcm.points[1:100]
for o in range(25):
    for p in range(15000):
        if (targets[o][p] == 8):
            pcr.add(points[o][p])


In [65]:
print(pcr.shape)

torch.Size([25, 0, 3])


In [61]:
# display true full point cloud
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(pcr.permute(2, 0, 1).reshape(3, -1).to('cpu').T)
pcd.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(targets.reshape(-1).to('cpu'))).T/255)

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 1 is not equal to len(dims) = 3

In [55]:
#display in window
o3.visualization.draw_geometries([pcd]) 

In [ ]:
# model inference for s3dis ref

points = points.to(DEVICE)

# Normalize each partitioned Point Cloud to (0, 1)
norm_points = points.clone()
norm_points = norm_points - norm_points.min(axis=1)[0].unsqueeze(1)
norm_points /= norm_points.max(axis=1)[0].unsqueeze(1)

with torch.no_grad():

    # prepare data
    norm_points = norm_points.transpose(2, 1)

    # run inference
    preds, _, _ = model(norm_points)

    # get metrics
    pred_choice = torch.softmax(preds, dim=2).argmax(dim=2)

In [ ]:
# display predicted full point cloud - in Open3d
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(points.permute(2, 0, 1).reshape(3, -1).to('cpu').T)
pcd.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(pred_choice.reshape(-1).to('cpu'))).T/255)

In [ ]:
#display in jupyter
draw(pcd)

In [ ]:
#display in window
o3.visualization.draw_geometries([pcd]) 

In [ ]:
# multi-order ransac
max_plane_idx = 8
pt_to_plane_dist = 0.02

segment_models = {}
segments = {}
rest = pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap('tab20')(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=1000)
    segments[i] = rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass", i, "/", max_plane_idx, "done. ")
    
#o3.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)], zoom=zoom, front=front, up=up, lookat=lookat)
#o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)] + [rest], zoom=zoom, front=front, up=up, lookat=lookat)

In [ ]:
rest.estimate_normals(
    search_param=o3.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [ ]:
o3.visualization.draw_geometries([rest],
                                  point_show_normal=True)

In [ ]:
for i in range(max_plane_idx):
    segments[i].estimate_normals(
    search_param=o3.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [ ]:
o3.visualization.draw_geometries([segments[1]],
                                  point_show_normal=True)

In [ ]:
o3.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)],
                                  point_show_normal=True)

In [ ]:
o3.visualization.draw_plotly([segments[i] for i in range(max_plane_idx)])

In [ ]:
o3.visualization.draw_plotly([segments[i] for i in range(max_plane_idx)] + [rest])

In [ ]:
o3.visualization.draw_plotly([rest])

## Segmenting cwc corridor

In [ ]:
# creating an empty open3d point cloud
#cwc_pcd = o3.geometry.PointCloud()

In [ ]:
#o3.io.read_point_cloud('../../data/cwc/geometry/cloud.ply', cwc_pcd)

In [ ]:
print("Load a ply point cloud, print it, and render it")
ply_point_cloud = o3.data.PLYPointCloud()
pcd = o3.io.read_point_cloud("../../data/cwc/geometry/mini_cloud.ply")
print(pcd)
print(np.asarray(pcd.points))
o3.visualization.draw_geometries([pcd])



In [ ]:
#torch import ply, doesn't work with model
#points = torch_geometric.io.read_ply("../../data/cwc/geometry/cloud.ply")

In [ ]:
np_points = np.asarray(pcd.points)

In [ ]:
print(len(np_points))

In [ ]:
#normalizing points: For some reason this breaks the cloud format
#points = self.normalize_points(points)
np_points = np_points - np_points.min(axis=0)
np_points /= np_points.max(axis=0)

In [ ]:
if len(np_points) > 120000:
    choice = np.random.choice(len(np_points), 120000, replace=False)
else:
    # case when there are less points than the desired number
    choice = np.random.choice(len(np_points), 120000, replace=True)
np_points = np_points[choice, :] 

In [ ]:
print(len(np_points))

In [ ]:
pt_points = torch.from_numpy(np_points).type(torch.float32)

In [ ]:
print(pt_points.shape)

In [ ]:
pt_points = pt_points.view(1, 120000, 3)

In [ ]:
# display predicted full point cloud - in Open3d
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(pt_points.permute(2, 0, 1).reshape(3, -1).to('cpu').T)
#pcd.points = o3.utility.Vector3dVector(pt_points.permute(2, 0, 1).reshape(3, -1).to('cpu').T)
#pcd.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(pred_choice.reshape(-1).to('cpu'))).T/255)

In [ ]:
#display in window
o3.visualization.draw_geometries([pcd]) 

In [ ]:
#points2 = pcd.points 
#doesn't work

In [ ]:
MODEL_PATH = './trained_models/seg_focal_dice_iou_rot/seg_model_68.pth'

model = PointNetSegHead(num_points=NUM_TEST_POINTS, m=NUM_CLASSES).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval();

In [ ]:
#reshaping tensor to fit model input
pt_points = pt_points[:60000] 
pt_points = pt_points.view(4, 15000, 3)

In [ ]:
#reshaping tensor to fit model input
pt_points = pt_points[:90000] 
pt_points = pt_points.view(4, 15000, 3)

In [ ]:
#reshaping tensor to fit model input - bigger version
pt_points = pt_points[:120000] 
pt_points = pt_points.view(8, 15000, 3)

In [ ]:
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(pt_points.permute(2, 0, 1).reshape(3, -1).to('cpu').T)
o3.visualization.draw_geometries([pcd])

In [ ]:
# model inference for cwc corridor

pt_points = pt_points.to(DEVICE)

# Normalize each partitioned Point Cloud to (0, 1)
norm_points = pt_points.clone()
norm_points = norm_points - norm_points.min(axis=1)[0].unsqueeze(1)
norm_points /= norm_points.max(axis=1)[0].unsqueeze(1)

with torch.no_grad():

    # prepare data
    norm_points = norm_points.transpose(2, 1)

    # run inference
    preds, _, _ = model(norm_points)

    # get metrics
    pred_choice = torch.softmax(preds, dim=2).argmax(dim=2)


In [ ]:
# display predicted full point cloud
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(pt_points.permute(2, 0, 1).reshape(3, -1).to('cpu').T)
pcd.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(pred_choice.reshape(-1).to('cpu'))).T/255)

draw(pcd)

### Segementing Saalastinsali

In [10]:
print("Load a ply point cloud, print it, and render it")
ply_point_cloud = o3.data.PLYPointCloud()
pcd = o3.io.read_point_cloud("../../data/saalasti/mesh/mesh.ply")
print(pcd)
print(np.asarray(pcd.points))
#o3.visualization.draw_geometries([pcd])

Load a ply point cloud, print it, and render it
PointCloud with 4540577 points.
[[  6.02087499 -11.96875     -0.94423194]
 [  6.         -11.99059806  -0.94445244]
 [  6.         -11.96875     -0.94951831]
 ...
 [-15.76822015  -2.7734375    0.9765625 ]
 [-15.7578125   -2.7734375    0.96941538]
 [-15.50271709  -3.5390625    0.9609375 ]]


In [11]:
np_points = np.asarray(pcd.points)

In [ ]:
#points = self.normalize_points(points)
np_points = np_points - np_points.min(axis=0)
np_points /= np_points.max(axis=0)

In [ ]:
print(len(np_points))

In [13]:
#Downsampling to 120000 points
if len(np_points) > 120000:
    choice = np.random.choice(len(np_points), 120000, replace=False)
else:
    # case when there are less points than the desired number
    choice = np.random.choice(len(np_points), 120000, replace=True)
np_points = np_points[choice, :] 

In [ ]:
#Downsampling to 60000 points
if len(np_points) > 60000:
    choice = np.random.choice(len(np_points), 60000, replace=False)
else:
    # case when there are less points than the desired number
    choice = np.random.choice(len(np_points), 60000, replace=True)
np_points = np_points[choice, :] 

In [14]:
pt_points = torch.from_numpy(np_points).type(torch.float32)

In [15]:
print(pt_points.shape)

torch.Size([120000, 3])


In [16]:
MODEL_PATH = './trained_models/seg_focal_dice_iou_rot/seg_model_68.pth'

model = PointNetSegHead(num_points=NUM_TEST_POINTS, m=NUM_CLASSES).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval();

In [ ]:
#reshaping tensor to fit model input - big version - requires much gpu memory
pt_points = pt_points[:4530000] 
pt_points = pt_points.view(302, 15000, 3)

In [ ]:
#reshaping tensor to fit model input - smål version
pt_points = pt_points[:150000] 
pt_points = pt_points.view(10, 15000, 3)

In [17]:
#reshaping tensor to fit model input - småller version
pt_points = pt_points[:120000] 
pt_points = pt_points.view(8, 15000, 3)

In [ ]:
#reshaping tensor to fit model input - smållest version
pt_points = pt_points[:60000] 
pt_points = pt_points.view(4, 15000, 3)

In [ ]:
#big batch alternative
MODEL_PATH = './trained_models/seg_focal_dice_iou_rot/seg_model_68.pth'

model = PointNetSegHead(num_points=60000, m=NUM_CLASSES).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval();

pt_points = pt_points[:120000] 
pt_points = pt_points.view(2, 60000, 3)

In [ ]:
#not sure if needed
#torch.cuda.empty_cache() # release GPU memory

In [18]:
# model inference for saalasti

pt_points = pt_points.to(DEVICE)

# Normalize each partitioned Point Cloud to (0, 1)
norm_points = pt_points.clone()
norm_points = norm_points - norm_points.min(axis=1)[0].unsqueeze(1)
norm_points /= norm_points.max(axis=1)[0].unsqueeze(1)

with torch.no_grad():

    # prepare data
    norm_points = norm_points.transpose(2, 1)

    # run inference
    preds, _, _ = model(norm_points)

    # get metrics
    pred_choice = torch.softmax(preds, dim=2).argmax(dim=2)


In [ ]:
print(pred_choice)

In [19]:
# display predicted full point cloud
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(pt_points.permute(2, 0, 1).reshape(3, -1).to('cpu').T)
pcd.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(pred_choice.reshape(-1).to('cpu'))).T/255)
o3.visualization.draw_geometries([pcd])
#draw(pcd)